In [ ]:
# Using google drive for saving model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflowjs

     |████████████████████████████████| 77 kB 4.7 MB/s 
     |████████████████████████████████| 462 kB 19.5 MB/s 


In [ ]:
# import
import tensorflow as tf
import numpy as np
import os
import tensorflowjs as tfjs

In [ ]:
path_to_file = "/content/drive/MyDrive/Final Project/Data/Friends_Transcript.txt"

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of Character {}'.format(len(text)))

Length of Character 4965384


In [ ]:
print(text[:250])

THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)
Written by: Marta Kauffman & David Crane
[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]
Monica: There's nothing to tell! He's just some guy I work with!
J


In [ ]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
print(vocab)

95 unique characters
['\n', '\r', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}']


In [ ]:
# Create a mapping from characters to numbers and vice versa
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print("{} is mapped to {}".format(text[:10], text_as_int[:10]))

THE ONE WH is mapped to [54 42 39  2 49 48 39  2 57 42]


In [ ]:
# Maximum sentence we are inputing to the RNN
seq_length =100
examples_per_epoch = len(text)//(seq_length+1)
print(examples_per_epoch)

49162


In [ ]:
# Creating dataset
# from_tensor_slices is like creating a generator for our dataset and is suitable for handling
# large datasets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
# .take is like iloc in pandas
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

T
H
E
 
O


In [ ]:
# .batch converts into batches of fixed size
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)\r\nWritten by: Marta Kauffman & D'
"avid Crane\r\n[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]\r\nMonica: There's not"
"hing to tell! He's just some guy I work with!\r\nJoey: C'mon, you're going out with the guy! There's go"
'tta be something wrong with him!\r\nChandler: All right Joey, be nice. So does he have a hump? A hump a'
"nd a hairpiece?\r\nPhoebe: Wait, does he eat chalk?\r\n(They all stare, bemused.)\r\nPhoebe: Just, 'cause, "


In [ ]:
def split_input_target(chunk):
    """
    This function generate input and target text from the given text.
    Input text does not contain last part and target doesnot contain first character
    """
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)\r\nWritten by: Marta Kauffman & '
Target data: 'HE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)\r\nWritten by: Marta Kauffman & D'


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 54 ('T')
  expected output: 42 ('H')
Step    1
  input: 42 ('H')
  expected output: 39 ('E')
Step    2
  input: 39 ('E')
  expected output: 2 (' ')
Step    3
  input: 2 (' ')
  expected output: 49 ('O')
Step    4
  input: 49 ('O')
  expected output: 48 ('N')


In [ ]:
# Now we need to shuffle the data and pack into batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
#Constants for model
vocab_size = len(vocab)
embedding_dim = 512
# Using half of rnn_units for LSTM
# Speed of training was reduced to half, so i can try 1024 units
rnn_units = 2048

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful= True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful= True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 95) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 512)           48640     
                                                                 
 lstm (LSTM)                 (64, None, 2048)          20979712  
                                                                 
 lstm_1 (LSTM)               (64, None, 2048)          33562624  
                                                                 
 dense (Dense)               (64, None, 95)            194655    
                                                                 
Total params: 54,785,631
Trainable params: 54,785,631
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Here we are chosing the next character randomly based on its probablity
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
#idk what this line does
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices


array([15, 80, 69, 77, 45, 32, 25, 36, 10, 64, 45, 92, 51,  3, 71, 49, 34,
       48, 51, 24, 75,  5, 27, 50, 77, 69, 45, 20,  3,  5, 86, 80, 17, 13,
       57, 58, 27, 69, 79, 32, 69, 32, 26, 61, 60, 68, 12, 66, 10, 39, 77,
       20, 70, 68, 78, 79, 65, 71, 30, 75, 85, 75, 54, 19, 36, 65,  1, 10,
       73, 68, 68, 45, 23, 19, 10, 55, 10, 45,  0, 75, 52, 36, 42, 79, 93,
       89, 53, 50, 84, 13,  0,  0, 24, 31, 55,  1, 83, 52, 47, 41])

In [ ]:
#Decoding what this means
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'grins as Ross removes the girlie jacket, grabs his own, and rushes out.)\r\n[Scene: The Lamaze class, '

Next Char Predictions: 
 '-odlK>7B(_K{Q!fO@NQ6j#9PldK2!#uo/+WX9dn>d>8[Zc*a(El2ecmn`f<jtjT1B`\r(hccK51(U(K\njRBHn|xSPs+\n\n6=U\rrRMG'


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
# model.load_weights('./content/training_checkpoints/ckpt_18')
import shutil
shutil.rmtree('/content/training_checkpoints')

FileNotFoundError: ignored

In [ ]:
 model.load_weights('/content/drive/MyDrive/Final Project/ckpt_20')

In [ ]:
EPOCHS = 2
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/2
 79/768 [==>...........................] - ETA: 3:37 - loss: 0.5069

KeyboardInterrupt: ignored

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('/content/drive/MyDrive/Final Project/ckpt_20')
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 512)            48640     
                                                                 
 lstm_6 (LSTM)               (1, None, 2048)           20979712  
                                                                 
 lstm_7 (LSTM)               (1, None, 2048)           33562624  
                                                                 
 dense_3 (Dense)             (1, None, 95)             194655    
                                                                 
Total params: 54,785,631
Trainable params: 54,785,631
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
  # Number of characters to generate
  num_generate = 3000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []
  temperature = 0.7

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
model.save('Model_LSTM-2-(512-2048-best-1).h5')

In [ ]:
print(generate_text(model, start_string=u"Monica: I know"))

Monica: I know. (Pause) Man, you missed it. Show you when I get home.
Eric: So, see I was really considering this morning and I never really liked you. Oh whats the matter?
Ross: Thats right, isnt it for? I mean if they had some money for this little guadandusOh my God. I mean shes out uh some places because I drank a death sure I gave her my schedule before.
Ross: Oh dude, we gotta get married here.
Chandler: I dont know. Its really scary man: (To Joey) Id like a Walking pizza T-shirt that says, Fossils are my friend. (To the policeman) Well, let me take those tomorrow.
Joey: (smiling and thighs) Oh yeah.
Rachel: Well thank you too.
Ross: Yeah. I missed you too.
Elizabeth: Oh my God, Ross! Youre amazing!
Ross: Im sorry, Im being so rubning the book! Okay? Maybe if I was in seven months (again so she realizes her pants)
Rachel: Okay. You want it. I'm gonna need your help. 
Ross: Ok. (they pull over, given the director) And that's how it's done! (Phoebe hits him and Ross reads it.) Hey 

In [ ]:
os.listdir('./training_checkpoints')

FileNotFoundError: ignored

In [ ]:
# There is some issue with downloading model.
from google.colab import files
files.download('Model_LSTM-2-(512-2048-ck25).h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os, tarfile
 
import os
from google.colab import files

def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()
 
  files.download(output_filename)
 
 
make_targz_one_by_one('WebModel_tar', '/content/ WebModel')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Saving the look up table for encoding and decoding characters
import json
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 '&': 7,
 "'": 8,
 '(': 9,
 ')': 10,
 '*': 11,
 '+': 12,
 ',': 13,
 '-': 14,
 '.': 15,
 '/': 16,
 '0': 17,
 '1': 18,
 '2': 19,
 '3': 20,
 '4': 21,
 '5': 22,
 '6': 23,
 '7': 24,
 '8': 25,
 '9': 26,
 ':': 27,
 ';': 28,
 '<': 29,
 '=': 30,
 '>': 31,
 '?': 32,
 '@': 33,
 'A': 34,
 'B': 35,
 'C': 36,
 'D': 37,
 'E': 38,
 'F': 39,
 'G': 40,
 'H': 41,
 'I': 42,
 'J': 43,
 'K': 44,
 'L': 45,
 'M': 46,
 'N': 47,
 'O': 48,
 'P': 49,
 'Q': 50,
 'R': 51,
 'S': 52,
 'T': 53,
 'U': 54,
 'V': 55,
 'W': 56,
 'X': 57,
 'Y': 58,
 'Z': 59,
 '[': 60,
 ']': 61,
 '^': 62,
 '_': 63,
 '`': 64,
 'a': 65,
 'b': 66,
 'c': 67,
 'd': 68,
 'e': 69,
 'f': 70,
 'g': 71,
 'h': 72,
 'i': 73,
 'j': 74,
 'k': 75,
 'l': 76,
 'm': 77,
 'n': 78,
 'o': 79,
 'p': 80,
 'q': 81,
 'r': 82,
 's': 83,
 't': 84,
 'u': 85,
 'v': 86,
 'w': 87,
 'x': 88,
 'y': 89,
 'z': 90,
 '{': 91,
 '|': 92,
 '}': 93}

In [ ]:
import json

with open('encodings.json', 'w') as fp:
    json.dump(char2idx, fp)